In [1]:
import json, sys, os, requests
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
colors = json.loads(
    requests.get(
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/colors/eco-colors.json"
    ).content
)

In [3]:
def get_ons_data_ts(dataset, series, save=True, api_path="https://api.ons.gov.uk"):
    r = requests.get(
        api_path
        + "/timeseries/"
        + series.lower()
        + "/dataset/"
        + dataset.lower()
        + "/data"
    )
    content = json.loads(r.content)
    if save:
        print("Saving DATASET", dataset, "SERIES", series, "to local mirror...")
        open("../../datasets/ons/" + dataset + "/" + series + ".json", "w").write(
            json.dumps(content)
        )
        print("OK!")
    return content

In [6]:
dataset = "lms"
data = {}
series = ["mgsx", "lf24", "kac3", "lf2s"]
series_colors = {"mgsx": "eco-pink", "lf24": "eco-turquiose", "kac3": "eco-yellow"}
series_offset = {"mgsx": 0.8, "lf24": 0.5, "kac3": 1}
series_domain = {"mgsx": [2, 12], "lf24": [65, 80], "kac3": [-2, 6]}
series_domain2 = {"mgsx": [3.5, 5.5], "lf24": [74, 77], "kac3": [-2, 6]}
series_titles = {"mgsx": "Unemployment", "lf24": "Employment", "kac3": "Wage growth"}
series_subtitles = {
    "mgsx": "Rate (%), age 16+ seasonally adjusted",
    "lf24": "Rate (%), age 16+ seasonally adjusted",
    "kac3": "Weekly earnings, three month average, y-o-y growth (%)",
}
series_text3 = {
    "mgsx": "year(datum.date)>=2021",
    "lf24": "year(datum.date)>=2021",
    "kac3": "year(datum.date)>=2021&month(datum.date)>=1",
}
for serie in series:
    d = get_ons_data_ts(dataset, serie)
    data[serie] = {
        "title": d["description"]["title"].split(")")[0] + ")",
        "data": d["months"],
    }

Saving DATASET lms SERIES mgsx to local mirror...
OK!
Saving DATASET lms SERIES lf24 to local mirror...
OK!
Saving DATASET lms SERIES kac3 to local mirror...
OK!


Now push datasets to GitHub.

In [9]:
for l in ["live", "eco", "local"]:
    layers = []
    for serie in series:
        if l == "live":
            url = (
                "https://api.allorigins.win/raw?url=https://api.ons.gov.uk/timeseries/"
                + serie
                + "/dataset/"
                + dataset
                + "/data"
            )
        else:
            url = (
                "https://raw.githubusercontent.com/EconomicsObservatory/ECOdataHUB/main/datasets/ons/"
                + dataset
                + "/"
                + serie
                + ".json"
            )
        if l == "local":
            data = alt.Data(
                url=url,
                format=alt.DataFormat(type="json", property="months"),
            )
        else:
            data = pd.DataFrame(json.loads(requests.get(url).content)["months"])
            data["date"] = pd.to_datetime(data["date"])
        print("Processing", serie, "for", l, "...")
        base = (
            alt.Chart(data)
            .transform_calculate(serie=repr(serie))
            .transform_calculate(label="datum.year+' '+datum.month")
            .transform_calculate(date="toDate(datum.year+' '+datum.month+' '+1)")
            .encode(x=alt.X("date:T", axis=alt.Axis(grid=True, title="")))
        )
        area = base.mark_area(
            interpolate="monotone",
            fillOpacity=0.7,
            stroke=colors["eco-gray"],
            strokeWidth=0.5,
            color=alt.Gradient(
                gradient="linear",
                stops=[
                    alt.GradientStop(color="white", offset=0),
                    alt.GradientStop(
                        color=colors[series_colors[serie]], offset=series_offset[serie]
                    ),
                ],
                x1=0.8,
                x2=1,
                y1=1,
                y2=0,
            ),
        ).encode(
            y=alt.Y(
                "value:Q",
                axis=alt.Axis(grid=True, title=""),
                # scale=alt.Scale(domain=series_domain[serie]),
            )
        )
        line = base.mark_line(color=colors[series_colors[serie]]).encode(
            y=alt.Y(
                "value:Q",
                axis=alt.Axis(grid=True, title=""),
                scale=alt.Scale(domain=series_domain2[serie]),
            )
        )
        # Create a selection that chooses the nearest point & selects based on x-value
        nearest = alt.selection(
            type="single", nearest=True, on="mouseover", fields=["date"], empty="none"
        )
        # Transparent selectors across the chart. This is what tells us
        # the x-value of the cursor
        selectors = (
            base.mark_point()
            .encode(
                opacity=alt.value(0),
            )
            .add_selection(nearest)
        )

        # Draw points on the line, and highlight based on selection
        points = area.mark_point().encode(
            opacity=alt.condition(nearest, alt.value(1), alt.value(0))
        )

        # Draw text labels near the points, and highlight based on selection
        text = area.mark_text(align="left", dx=-25, dy=35).encode(
            text=alt.condition(nearest, "value:N", alt.value(" "))
        )

        # Draw text labels near the points, and highlight based on selection
        text2 = area.mark_text(align="right", dx=5, dy=10,).encode(
            text=alt.condition(nearest, "label:N", alt.value(" ")),
            x=alt.value(390),
            y=alt.value(0),
        )

        # Draw text labels near the points, and highlight based on selection
        text3 = line.mark_text(align="right", dx=5, dy=10, fontSize=13).encode(
            text=alt.condition(series_text3[serie], "value:Q", alt.value(" ")),
            x=alt.value(92),
            y=alt.value(3),
        )

        # Draw a rule at the location of the selection
        rules = base.mark_rule(color=colors["eco-gray"]).transform_filter(nearest)

        # Put the five layers into a chart and bind the data
        layer1 = alt.layer(area, selectors, points, rules, text, text2,).properties(
            width=400,
            height=150,
            title={"text": series_titles[serie], "subtitle": series_subtitles[serie]},
        )
        layer2 = (
            alt.layer(
                line,
                selectors,
                points,
                rules,
                text,
                text3,
            )
            .properties(width=100, height=150)
            .transform_filter("datum.date>='2020'")
        )
        layers.append(alt.hconcat(layer1, layer2, spacing=20))

    layers = alt.vconcat(
        layers[0], layers[1], layers[2]
    )  # .configure_view(stroke=None)
    layers.save(dataset + "_" + l + ".json")
    print("OK!")

Processing mgsx for live ...
Processing lf24 for live ...
Processing kac3 for live ...
OK!
Processing mgsx for eco ...
Processing lf24 for eco ...
Processing kac3 for eco ...
OK!
Processing mgsx for local ...
Processing lf24 for local ...
Processing kac3 for local ...
OK!


In [10]:
layers

alt.VConcatChart(...)

In [21]:
dataset = "lms"
data = {}
series = ["lf2s", "ybtm", "lf2t"]
series_colors = {"lf2s": "eco-pink", "ybtm": "eco-turquiose", "lf2t": "eco-yellow"}
series_offset = {"lf2s": 0.8, "ybtm": 0.5, "lf2t": 1}
series_domain = {"lf2s": [2, 12], "ybtm": [65, 80], "lf2t": [-2, 6]}
series_domain2 = {"lf2s": [20, 22], "ybtm": [15, 18], "lf2t": [24, 25]}
series_titles = {
    "lf2s": "Inactivity Rate",
    "ybtm": "Inactivity Rate",
    "lf2t": "Inactivity Rate",
}
series_subtitles = {
    "lf2s": "(%), All",
    "ybtm": "(%), Men",
    "lf2t": "(%), Women",
}
series_text3 = {
    "lf2s": "year(datum.date)>=2021",
    "ybtm": "year(datum.date)>=2021",
    "lf2t": "year(datum.date)>=2021&month(datum.date)>=1",
}
for serie in series:
    d = get_ons_data_ts(dataset, serie)
    data[serie] = {
        "title": d["description"]["title"].split(")")[0] + ")",
        "data": d["months"],
    }

Saving DATASET lms SERIES lf2s to local mirror...
OK!
Saving DATASET lms SERIES ybtm to local mirror...
OK!
Saving DATASET lms SERIES lf2t to local mirror...
OK!


In [23]:
for l in ["live","eco","local"]:
    layers = []
    for serie in series:
        if l == "live":
            url = (
                "https://api.allorigins.win/raw?url=https://api.ons.gov.uk/timeseries/"
                + serie
                + "/dataset/"
                + dataset
                + "/data"
            )
        else:
            url = (
                "https://raw.githubusercontent.com/EconomicsObservatory/ECOdataHUB/main/datasets/ons/"
                + dataset
                + "/"
                + serie
                + ".json"
            )
        if l == "local":
            data = alt.Data(
                url=url,
                format=alt.DataFormat(type="json", property="months"),
            )
        else:
            data = pd.DataFrame(json.loads(requests.get(url).content)["months"])
            data["date"] = pd.to_datetime(data["date"])
        print("Processing", serie, "for", l, "...")
        base = (
            alt.Chart(data)
            .transform_calculate(serie=repr(serie))
            .transform_calculate(label="datum.year+' '+datum.month")
            .transform_calculate(date="toDate(datum.year+' '+datum.month+' '+1)")
            .encode(x=alt.X("date:T", axis=alt.Axis(grid=True, title="")))
        )
        area = base.mark_area(
            interpolate="monotone",
            fillOpacity=0.7,
            stroke=colors["eco-gray"],
            strokeWidth=0.5,
            color=alt.Gradient(
                gradient="linear",
                stops=[
                    alt.GradientStop(color="white", offset=0),
                    alt.GradientStop(
                        color=colors[series_colors[serie]], offset=series_offset[serie]
                    ),
                ],
                x1=0.8,
                x2=1,
                y1=1,
                y2=0,
            ),
        ).encode(
            y=alt.Y(
                "value:Q",
                axis=alt.Axis(grid=True, title=""),
                # scale=alt.Scale(domain=series_domain[serie]),
            )
        )
        line = base.mark_line(color=colors[series_colors[serie]]).encode(
            y=alt.Y(
                "value:Q",
                axis=alt.Axis(grid=True, title=""),
                scale=alt.Scale(domain=series_domain2[serie]),
            )
        )
        # Create a selection that chooses the nearest point & selects based on x-value
        nearest = alt.selection(
            type="single", nearest=True, on="mouseover", fields=["date"], empty="none"
        )
        # Transparent selectors across the chart. This is what tells us
        # the x-value of the cursor
        selectors = (
            base.mark_point()
            .encode(
                opacity=alt.value(0),
            )
            .add_selection(nearest)
        )

        # Draw points on the line, and highlight based on selection
        points = area.mark_point().encode(
            opacity=alt.condition(nearest, alt.value(1), alt.value(0))
        )

        # Draw text labels near the points, and highlight based on selection
        text = area.mark_text(align="left", dx=-25, dy=35).encode(
            text=alt.condition(nearest, "value:N", alt.value(" "))
        )

        # Draw text labels near the points, and highlight based on selection
        text2 = area.mark_text(align="right", dx=5, dy=10,).encode(
            text=alt.condition(nearest, "label:N", alt.value(" ")),
            x=alt.value(390),
            y=alt.value(0),
        )

        # Draw text labels near the points, and highlight based on selection
        text3 = line.mark_text(align="right", dx=5, dy=10, fontSize=13).encode(
            text=alt.condition(series_text3[serie], "value:Q", alt.value(" ")),
            x=alt.value(92),
            y=alt.value(3),
        )

        # Draw a rule at the location of the selection
        rules = base.mark_rule(color=colors["eco-gray"]).transform_filter(nearest)

        # Put the five layers into a chart and bind the data
        layer1 = alt.layer(area, selectors, points, rules, text, text2,).properties(
            width=400,
            height=150,
            title={"text": series_titles[serie], "subtitle": series_subtitles[serie]},
        )
        layer2 = (
            alt.layer(
                line,
                selectors,
                points,
                rules,
                text,
                text3,
            )
            .properties(width=100, height=150)
            .transform_filter("datum.date>='2020'")
        )
        layers.append(alt.hconcat(layer1, layer2, spacing=20))

    layers = alt.vconcat(
        layers[0], layers[1], layers[2]
    )  # .configure_view(stroke=None)
    layers.save(dataset + "2_" + l + ".json")
    print("OK!")
layers

Processing lf2s for live ...
Processing ybtm for live ...
Processing lf2t for live ...
OK!
Processing lf2s for eco ...
Processing ybtm for eco ...
Processing lf2t for eco ...
OK!
Processing lf2s for local ...
Processing ybtm for local ...
Processing lf2t for local ...
OK!


alt.VConcatChart(...)